## How To Perform an Amplitude Fine Experiment

### Prerequisites
This guide assumes you have a configured `DeviceSetup` as well as `Qubit` objects with assigned parameters. Please see these guides (add links) if you need to create your setup and qubits for the first time. However, you can also run this notebook "as is" using an emulated session. If you are just getting started with the LabOne Q Applications Library, please don't hesitate to reach out to us at info@zhinst.com.

### Background
In this how-to guide, you'll optimize the pulse amplitude of the pi-pulse for a given qubit transition using the `amplitude_fine` experiment included in the LabOne Q Applications Library. It is recommended to have the pulse amplitude already roughly calibrated, e.g. with the `amplitude_rabi` experiment.

The amplitude of a pulse can be precisely calibrated using error amplifying gate sequences. These gate sequences apply the same quantum operation (qop) a variable number of times. Therefore, if each qop has a small error $\theta$ in the rotation angle then a sequence of multiple qop's will have a rotation error of $\text{reps}\cdot \theta$, where $\text{reps}$ is the number of repetitions. By implementing sequences with increasing number of repetitions we can obtain a correction value for the imperfect pulse amplitude (see also https://qiskit-community.github.io/qiskit-experiments/tutorials/calibrations). A larger number of repetitions leads to a higher precision in the pulse calibration, finally limited by the coherence time of the qubit.  

![](../../../images/amplitude_fine.svg "Demonstration of pulses and the corresponding rotation around the Bloch sphere during an amplitude fine experiment.")

In the `amplitude_fine` experiment, we add an x90 gate before the gates that are repeated, in order to move the ideal population to the equator of the Bloch sphere where the sensitivity to over/under rotations is the highest. 

![](../../../images/amplitude_fine_underrotation.svg "Pulses with too small ammplitudes cause under-rotation.")

Below we show the results of an amplitude-fine experiment for calibration an x180 operation. The ping-pong pattern in the figure on the left indicates an under-rotation which makes the initial state rotate less than $\pi$. The rotated phase per gate $\phi_{\text{x180}}$ is given by the frequency of the fitted oscillation. The comparison to the ideal angle defines the correction factor $c$

$c=\frac{\phi_{\text{x180}}}{\pi}$

Finally, we scale our amplitude by $c^{-1}$ to obtain the optimized amplitude. 



### Imports

You'll start by importing the amplitude-Rabi experiment from `laboneq_applications`, as well as `laboneq.simple` and a demo QPU and device setup to run in emulation mode.

In [ ]:
import numpy as np
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

from laboneq_applications.experiments import amplitude_fine
from laboneq_applications.qpu_types.tunable_transmon.demo_qpus import demo_platform

### QPU and Device Setup

You'll generate six qubits with pre-defined parameters, as well as a `Device_Setup` consisting of a SHFQC+, HDAWG, and PQSC. If you already have your own `DeviceSetup` and qubits configured, you'll instead initialize the session using your setup.

In [ ]:
my_platform = demo_platform(6)

Then, you'll connect to the `Session`. Here we connect to an emulated one:

In [ ]:
session = Session(my_platform.setup)
session.connect(do_emulation=True)

### Running the Amplitude Fine x180 Workflow

You'll now make the experiment workflow and run:

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
)

my_results = my_workflow.run()

In the `amplitude_fine` experiment for an x180 gate, we apply the quantum operation `x180` multiple times, defined by the `repetitions`. 

#### Output Simulation

You can also inspect the compiled experiment and plot the simulated output:

In [ ]:
compiled_exp = my_results.tasks["compile_experiment"].output
plot_simulation(compiled_exp, length=12e-6)

#### Inspecting the Source Code

You can inspect the source code of the `create_experiment` task defined in `amplitude_fine` to see how the experiment pulse sequence is created:

In [ ]:
print(amplitude_fine.create_experiment.src)

### Changing the Options

We can give our `amplitude_fine` experiment options. First, inspect what they currently are:

In [ ]:
my_new_opts = amplitude_fine.experiment_workflow.options()
my_new_opts

Then provide new options. This time, you'll enable the calibration traces and change the counts.

In [ ]:
my_new_opts.count(2048)
my_new_opts.use_cal_traces(True)
my_new_opts

#### Run the workflow with updated options

Now, run the workflow with new options and inspect the simulated output. You'll notice that the calibration traces are no longer there.

In [ ]:
my_new_workflow = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_exp = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_exp, length=50e-6)

### Temporary settings

The qubit parameters are used to control the settings of pulses and instruments during the experiment. We can run the amplitude-Rabi experiment with different settings by passing it a copy of the qubits with modified parameters:

In [ ]:
# Make a copy of the qubits
temp_qubits = my_platform.qpu.copy_qubits()
# Change the length of the drive pulses
temp_qubits[0].parameters.ge_drive_length = 1000e-9
temp_qubits[1].parameters.ge_drive_length = 1000e-9

my_new_workflow = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=my_platform.qpu,
    qubits=[temp_qubits[0], temp_qubits[1]],  # pass temporary qubits
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
    options=my_new_opts,
)

my_new_results = my_new_workflow.run()
new_compiled_exp = my_new_results.tasks["compile_experiment"].output
plot_simulation(new_compiled_exp, length=10e-6)

### Running the Amplitude Fine x90 Workflow

We can use the same concept to optimize other quantum operations, e.g. the amplitude of the `x90` quantum operation.

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits

my_workflow = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    repetitions=[[1,2,3,4,5], [1,2,3,4,5]],
)

my_results = my_workflow.run()

Note that `reps` is now in steps of two: As mentioned in the introduction, the sensitivity to over/under rotations is the highest for a final population at the equator of the Bloch sphere. 

### Running the general Amplitude Fine Workflow

Besides standard operations, we can use the same concept to optimize quantum operations in general. Define your own quantum operation and measure deviations from the ideal `target_angle` using the `amplitude_fine` experiment. 

In [ ]:
# our qubits live here in the demo setup:
qubits = my_platform.qpu.qubits
my_qop="y90"

my_workflow = amplitude_fine.experiment_workflow(
    session=session,
    qpu=my_platform.qpu,
    qubits=[qubits[0], qubits[1]],
    amplification_qop=my_qop,
    target_angle=np.pi/2,
    phase_offset=-np.pi/2,
    repetitions=[[1,2,3,4], [1,2,3,4]],
)

my_results = my_workflow.run()

Great! You've now run your amplitude fine experiment. Check out these other experiments to keep characterizing your qubits:

In [ ]:
# TODO: Add experiment links
# TODO: Add Analysis